[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MeteoSwiss/inca-examples/blob/main/notebooks/plot_RR_cartopy.ipynb)

# Plot INCA data

In this example, we will show how to import and plot a precipitation forecast from the INCA [nowcasting system](https://www.meteoswiss.admin.ch/home/measurement-and-forecasting-systems/warning-and-forecasting-systems/nowcasting.html).

## Set-up Colab environment

**Important**: In colab, execute this section one cell at a time. Trying to excecute all the cells at once may results in cells being skipped and some dependencies not being installed.

First, let's set up our working environment. Note that these steps are only needed to work with google colab. 

#### Install dependencies

Now, let's install the  dependendies that will allow us to plot and read the example data.
- xarray: read the netCDF4 dataset
- matplotlib: visualization

In [ ]:
!pip install xarray matplotlib

## Upload the data

Now that we have the environment ready, let's upload an example dataset from your local file system.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

## Read the data

Now that we have uploaded the example data, let's import the example netCDF with xarray's `open_dataset` method.

We import a 6-hour precipitation nowcast, which corresponds to a sequence of 37 frames of 2-D precipitation composites (1 analysis and 36 forecast lead times).

In [ ]:
import xarray as xr

ds = xr.open_dataset("RR_INCA.nc")
ds

## Basic plotting

Xarray's includes some plotting functionalities that we can use to visualize our example data.

In [ ]:
precip_field = ds.RR.isel(time=0) # select the first rainfall field, that is, the analysis
precip_field.plot()

One can easily add transparency to the dry areas and customize the [colormap](https://matplotlib.org/stable/tutorials/colors/colormaps.html).

In [ ]:
import numpy as np

levels = np.logspace(-1, 2, 10, base=10)
cmap = "Blues"
precip_field.where(precip_field >= 0.1).plot(levels=levels, cmap=cmap)

## Plot with Cartopy

[Cartopy](https://scitools.org.uk/cartopy/docs/latest/) is a python library for plotting georeferenced datasets and maps. 

Installing Cartopy is a bit more involved as it requires few additonal library dependencies. Also, shapely need to be reinstalled by ignoring the binary
wheels to make it compatible with Cartopy.

In [ ]:
!apt-get install -qq libgdal-dev libgeos-dev
!pip uninstall --yes shapely
!pip install shapely --no-binary shapely
!pip install cartopy pyepsg

We first define a method to plot reference geographical features to an existing figure axis.

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

def add_geo_features(ax):
    """
    Add reference geographical layers such as thenational boundaries
    and the main lakes and rivers.
    """
    ax.add_feature(
        cfeature.NaturalEarthFeature(
            "cultural",
            "admin_0_boundary_lines_land",
            scale="10m",
            edgecolor="black",
            facecolor="none",
            linewidth=1,
        )
    )
    ax.add_feature(
        cfeature.NaturalEarthFeature(
            "physical",
            "rivers_lake_centerlines",
            scale="10m",
            edgecolor=np.array([0.59375, 0.71484375, 0.8828125]),
            facecolor="none",
            linewidth=1,
        )
    )

To use Cartopy, we need to specify the source and target coordinate reference system. In our case, the Swiss coordinate system CH1903/LV03 can be identified by its EPSG code [21781]((https://epsg.io/21781)).

In [ ]:
crs = ccrs.epsg(21781)

# plot the precipitation field on the original Swiss projection
ax = precip_field.where(precip_field > 0).plot(
    transform=crs,
    subplot_kws=dict(projection=crs),
    levels=levels, 
    cmap=cmap
    ).axes
add_geo_features(ax)

With Cartopy, we can easily switch to alternative geographical [projections](https://scitools.org.uk/cartopy/docs/latest/crs/projections.html), here for example using Plate Carrée.

In [ ]:
# plot the precipitation field on the Plate Carrée projection
ax = precip_field.where(precip_field > 0).plot(
    transform=crs,
    subplot_kws=dict(projection=ccrs.PlateCarree()),
    levels=levels, 
    cmap=cmap
    ).axes
add_geo_features(ax)

# draw lat lon lines
grid_lines = ax.gridlines(
    crs=ccrs.PlateCarree(), 
    draw_labels=True, 
    dms=True
)
grid_lines.top_labels = grid_lines.right_labels = False
grid_lines.y_inline = grid_lines.x_inline = False
grid_lines.rotate_labels = False

# zoom on the map
ax.set_extent([5.5, 11, 44.5, 49])